In [1]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
from sklearn.model_selection import train_test_split

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

## Załadowanie danych oczyszczonych i zlematyzowanych oraz dalsze czyszczenie

In [2]:
dane = pd.read_csv('00_dane/dane_TM.csv',  sep='|', encoding='UTF-8')

In [10]:
dane.head()

,date_,speech_rawer,full_name,party_final,speech_rawest
0,01-02-1992,Chciałbym kilka zdań powiedzieć na temat skut...,Aleksander Bentkowski,Polskie Stronnictwo Ludowe,"[chcieć, zdanie, powiedzieć, temat, skutek, us..."
1,01-02-1992,"Przysłuchując się tej bardzo emocjonalnej, dł...",Aleksander Kwaśniewski,Sojusz Lewicy Demokratycznej,"[przysłuchiwać, emocjonalny, długo, trwać, deb..."
2,01-02-1992,"Różne przebyliśmy drogi, żeby tu dzisiaj powi...",Alojzy Pietrzyk,Klub Parlamentarny NSZZ Solidarność,"[różny, przebyć, droga, powiedzieć, swój, ocze..."
3,01-02-1992,"Wszystkim nam bardzo zależy na tym, aby urato...",Alojzy Szablewski,Klub Parlamentarny Zjednoczenia Chrześcijańsko...,"[wszystek, zależeć, uratować, przemysł, stocze..."
4,01-02-1992,Z brzmienia pkt. 1 projektu uchwały odczytuję...,Andrzej Baraniecki,Sojusz Lewicy Demokratycznej,"[brzmię, punkt, projekt, uchwała, odczytować, ..."


In [3]:
%%time
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub('[\W\d]',' ', x.lower()))
nlp = spacy.load('pl_model')
stop = set(nlp.Defaults.stop_words)
stop.add('zł')
stop.add('pkt')
stop.add('art')
stop.add('ustawy')
stop.add('r')
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split() if word not in stop])
#czas dla apply : 19s

<unknown>:1: DeprecationWarning: invalid escape sequence \W


Wall time: 54.6 s


In [4]:
len(dane)

158879

In [12]:
dane.to_csv('00_dane/dane_TM.csv', sep='|', encoding='UTF-8', index=False)


## LDA oparte o BOW
Szukanie parametrów

### Modyfikacje parametrów słownika

Liczba tematów - 20, odcięcie 50%

In [26]:
bow_30_50_20 = funcs.print_topics(lda_model.print_topics(-1))
pickle.dump( bow_30_50_20, open( "bow_30_50_20.p", "wb" ) )

In [4]:
test1, test2 = train_test_split(dane['speech_rawest'],test_size=0.5)   

In [18]:
test1.size

79439

In [27]:
start = timer()
dictionary = gensim.corpora.Dictionary(test1)
dictionary.filter_extremes(no_above=0.3, no_below=50)
print(len(dictionary))
BOW = [dictionary.doc2bow(speech) for speech in test1]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=35, passes=4, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))
#funcs.print_topics(lda_model.print_topics(-1))

13087
0:05:03.269917


In [28]:
%%time
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=test1, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
#Coherence Score:  0.5810685000684818


Coherence Score:  0.5774677719876303
Wall time: 6min 56s


In [ ]:
30% bazy
Coherence Score:  0.4452242642001402
Wall time: 2min 50s
Coherence Score:  0.4905338569697743
Wall time: 5min 34s
    Coherence Score:  0.48843461542030786
Wall time: 6min 16s

In [59]:
start = timer()
no_a = [0.3, 0.4, 0.5]
num_t = [20, 30, 40, 50]
p = [2, 3, 4 ]
res = pd.DataFrame()

for n_a in no_a:
    for n_t in num_t:
        for pas in p:
            dictionary = gensim.corpora.Dictionary(test1)
            dictionary.filter_extremes(no_above=n_a, no_below=50)
            BOW = [dictionary.doc2bow(speech) for speech in test1]
            tfidf = models.TfidfModel(BOW)
            corpus_tfidf = tfidf[BOW]

            lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=n_t, passes=pas, workers=2)
            coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=test1, dictionary=dictionary, coherence='c_v')
            coherence_lda = coherence_model_lda.get_coherence()
            print(n_a, n_t, pas, coherence_lda)
            res = res.append([[n_a, n_t, pas, coherence_lda]], ignore_index=True)
end = timer()
print(timedelta(seconds=timer()-start))            

0.3 20 2 0.5149798944439221
0.3 20 3 0.5639406582361796
0.3 20 4 0.5326794004692024
0.3 30 2 0.5484667091331878
0.3 30 3 0.5487364866053941
0.3 30 4 0.550028643669885
0.3 40 2 0.559127482613352
0.3 40 3 0.5681251529472859
0.3 40 4 0.5870566726707523
0.3 50 2 0.5420331235703096
0.3 50 3 0.5696529330100686
0.3 50 4 0.5804739702712741
0.4 20 2 0.49728752138048016
0.4 20 3 0.49658278029627273
0.4 20 4 0.5167402962257502
0.4 30 2 0.48898336985392915
0.4 30 3 0.5507258617448654
0.4 30 4 0.5325844634047786
0.4 40 2 0.5143183313744835
0.4 40 3 0.5419418407447472
0.4 40 4 0.5592280670785438
0.4 50 2 0.5297645884404241
0.4 50 3 0.5429406063469867
0.4 50 4 0.5581057494025395
0.5 20 2 0.4523112987327055
0.5 20 3 0.4697317469932886
0.5 20 4 0.49945740327560967
0.5 30 2 0.4503719634423547
0.5 30 3 0.4862242999099747
0.5 30 4 0.5208762496160027
0.5 40 2 0.4809994305685823
0.5 40 3 0.5182689628821622
0.5 40 4 0.5292198474359466
0.5 50 2 0.4817094952171486
0.5 50 3 0.525863586091386
0.5 50 4 0.54082718

In [7]:
start = timer()
no_a = [0.1, 0.2]
num_t = [40]
p = [5, 7, 10]
res = pd.DataFrame()

for n_a in no_a:
    for n_t in num_t:
        for pas in p:
            dictionary = gensim.corpora.Dictionary(test1)
            dictionary.filter_extremes(no_above=n_a, no_below=50)
            BOW = [dictionary.doc2bow(speech) for speech in test1]
            tfidf = models.TfidfModel(BOW)
            corpus_tfidf = tfidf[BOW]

            lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=n_t, passes=pas, workers=2)
            coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=test1, dictionary=dictionary, coherence='c_v')
            coherence_lda = coherence_model_lda.get_coherence()
            print(n_a, n_t, pas, coherence_lda)
            res = res.append([[n_a, n_t, pas, coherence_lda]], ignore_index=True)
end = timer()
print(timedelta(seconds=timer()-start))            

0.1 40 5 0.6575481881825509
0.1 40 7 0.6401501603737284
0.1 40 10 0.6563124893641243
0.2 40 5 0.6160356204422952
0.2 40 7 0.6212566643224987
0.2 40 10 0.6213075624555616
1:33:46.041675


TO DO:

Zapisać wyniki dla najlepszego modelu

In [60]:
pickle.dump( res, open( "res.p", "wb" ) )

In [19]:
pyLDAvis.enable_notebook()


In [28]:
vis = pyLDAvis.gensim.prepare(lda_model, BOW, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.022653 -0.096296       1        1  10.080772
20    -0.272737 -0.069813       2        1   6.804621
17    -0.183145 -0.170149       3        1   5.596610
12    -0.140322 -0.211647       4        1   4.966975
26     0.149210 -0.156835       5        1   4.594864
0      0.207117  0.018561       6        1   4.593047
15    -0.116390  0.078204       7        1   3.997863
23     0.197290 -0.055827       8        1   3.801178
29    -0.040199  0.149786       9        1   3.642122
21     0.157094 -0.043537      10        1   3.124763
1     -0.060509  0.139538      11        1   3.018207
6      0.073092  0.064634      12        1   2.984050
31    -0.069416  0.058262      13        1   2.679852
18    -0.265928 -0.045419      14        1   2.671003
4     -0.134921 -0.146688      15        1   2.551633
14     0.085009  0.017390      16        1   2.395110
22    -0.022445  0.125080      17        1   2.313910
16     0.008893  0.085799      18        1   2.200474
3      0.054795 -0.098094      19        1   2.183180
32    -0.034709  0.008312      20        1   2.174885
28    -0.048594  0.129781      21        1   1.958175
8      0.144006 -0.167859      22        1   1.937464
27    -0.053995  0.168367      23        1   1.895893
7      0.059172  0.014509      24        1   1.826554
2     -0.003778  0.084650      25        1   1.816259
25     0.072862 -0.109064      26        1   1.680281
11     0.101727  0.085247      27        1   1.673463
33     0.058787  0.103334      28        1   1.597402
19     0.066353  0.083352      29        1   1.558601
9      0.079888 -0.033814      30        1   1.468606
10    -0.098589  0.052969      31        1   1.386718
24    -0.032901  0.092502      32        1   1.376703
13     0.020178  0.021381      33        1   1.272677
34     0.081157 -0.133685      34        1   1.211633
30    -0.015397 -0.042932      35        1   0.964458, topic_info=             Term           Freq          Total Category  logprob  loglift
111      poprawka  125911.000000  125911.000000  Default  30.0000  30.0000
754    europejski  114010.000000  114010.000000  Default  29.0000  29.0000
828          unia   98584.000000   98584.000000  Default  28.0000  28.0000
4734      podatek   62188.000000   62188.000000  Default  27.0000  27.0000
1353      dziecko   55876.000000   55876.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
275    informacja    1901.649780   60838.519531  Topic35  -5.1868   1.1759
1414       środek    1960.167236   96184.773438  Topic35  -5.1565   0.7481
5384    nadzorczy    1091.861694    6334.749512  Topic35  -5.7416   2.8832
2128  działalność    1143.495361   59294.054688  Topic35  -5.6954   0.6929
470     działanie    1095.294434   71580.367188  Topic35  -5.7385   0.4616

[2711 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
8692     35  0.999254     abonament
8830     35  0.998800  abonamentowy
8518     10  0.006473       abonent
8518     35  0.990325       abonent
2232      3  0.028232       aborcja
...     ...       ...           ...
3951     15  0.007406       żywność
3951     17  0.011295       żywność
3951     21  0.313662       żywność
3951     23  0.004259       żywność
3951     27  0.600660       żywność

[19253 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 21, 18, 13, 27, 1, 16, 24, 30, 22, 2, 7, 32, 19, 5, 15, 23, 17, 4, 33, 29, 9, 28, 8, 3, 26, 12, 34, 20, 10, 11, 25, 14, 35, 31])

In [80]:
bow_30_50_50 = funcs.print_topics(lda_model.print_topics(-1))

## LDA oparte o TF-IDF

In [8]:
start = timer()
no_a = [0.1, 0.2, 0.3, 0.4, 0.5]
num_t = [30, 40, 50]
p = [3, 5, 7, 10]
res = pd.DataFrame()

for n_a in no_a:
    for n_t in num_t:
        for pas in p:
            dictionary = gensim.corpora.Dictionary(test1)
            dictionary.filter_extremes(no_above=n_a, no_below=50)
            BOW = [dictionary.doc2bow(speech) for speech in test1]
            tfidf = models.TfidfModel(BOW)
            corpus_tfidf = tfidf[BOW]

            lda_model = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary,num_topics=n_t, passes=pas, workers=2)
            coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=test1, dictionary=dictionary, coherence='c_v')
            coherence_lda = coherence_model_lda.get_coherence()
            print(n_a, n_t, pas, coherence_lda)
            res = res.append([[n_a, n_t, pas, coherence_lda]], ignore_index=True)
end = timer()
print(timedelta(seconds=timer()-start))  

0.1 30 3 0.5877595421644253
0.1 30 5 0.5975840924498675
0.1 30 7 0.5857381209803806
0.1 30 10 0.5999860865135607
0.1 40 3 0.5465898033747111
0.1 40 5 0.5201665874485345
0.1 40 7 0.5934270978562078
0.1 40 10 0.6115890373470071
0.1 50 3 0.5598165636963993
0.1 50 5 0.5643334105338714
0.1 50 7 0.5825017282833185
0.1 50 10 0.5807643984612598
0.2 30 3 0.5543915967765158
0.2 30 5 0.562164801582098
0.2 30 7 0.5965294648606458
0.2 30 10 0.5781703960384689
0.2 40 3 0.559269025071551
0.2 40 5 0.5511482785510295
0.2 40 7 0.5822837287894106
0.2 40 10 0.5946897555874026
0.2 50 3 0.5392970600309345
0.2 50 5 0.5744857040128464
0.2 50 7 0.5872166661894037
0.2 50 10 0.5721321339274452
0.3 30 3 0.5649982399003852
0.3 30 5 0.5784432693205408
0.3 30 7 0.5973921764873271
0.3 30 10 0.5703566451293612
0.3 40 3 0.5447602510067644
0.3 40 5 0.5621630827975151
0.3 40 7 0.5952370954737324
0.3 40 10 0.6094386644296848
0.3 50 3 0.5463186507509844
0.3 50 5 0.5700782748396733
0.3 50 7 0.5382656465937035
0.3 50 10 0.55

In [100]:
funcs.print_topics(lda_model_tfidf.print_topics(-1))

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,złoty,rolnik,rząd,dziecko,poprawka
1,budżet,rolnictwo,polska,praca,senat
2,podatek,rolny,premier,osoba,klub
3,mld,europejski,marszałek,zdrowie,zmiana
4,rząd,unia,człowiek,zmiana,przyjąć
5,bank,poprawka,sejm,poprawka,głosować
6,wzrost,rynek,prosić,służba,sprawozdanie
7,budżetowy,rozwój,polityczny,sąd,nieruchomość
8,pieniądz,podatek,powiedzieć,świadczenie,zgłosić
9,wydatek,gospodarstwo,polak,rodzina,konwencja


In [ ]:
start = timer()

dictionary = gensim.corpora.Dictionary(dane['speech_rawest'])
dictionary.filter_extremes(no_above=0.3, no_below=50)
BOW = [dictionary.doc2bow(speech) for speech in dane['speech_rawest']]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary,num_topics=30, passes=5, workers=2)
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=dane['speech_rawest'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(n_a, n_t, pas, coherence_lda)
           
end = timer()
print(timedelta(seconds=timer()-start))  
pickle.dump(lda_model, open("lda_tfidf_model_2.p", "wb"))

In [10]:
pickle.dump(lda_model, open("lda_tfidf_model.p", "wb"))